# Aprendizaje Multietiqueta de Patrones Geométricos en Objetos de Herencia Cultural
# Kunisch Features from ResNet architectures
## Seminario de Tesis II, Primavera 2022
### Master of Data Science. Universidad de Chile.
#### Prof. guía: Benjamín Bustos - Prof. coguía: Iván Sipirán
#### Autor: Matías Vergara

El objetivo de este notebook es extraer features mediante ResNet

## Imports

In [1]:
root_dir = '..'

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import numpy as np
%matplotlib inline
from matplotlib import pyplot as plt
from torchvision import datasets, models, transforms
import time
import os
import copy
import pandas as pd
import math
import random
import shutil

from torch.utils.data import Dataset
from PIL import Image

from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader
import numpy as np, scipy.io
import argparse
import json

## Configuración de dispositivo

In [3]:
CUDA_ID = 0

device = torch.device(f'cuda:{CUDA_ID}' if torch.cuda.is_available() else 'cpu')
print(f"Usando device: {torch.cuda.get_device_name(device)}")

Usando device: NVIDIA GeForce RTX 3090


## Configuración de datos y modelo

In [4]:
# Flags para los datos sintéticos
# Cada flag está asociada a una o más funciones de data augmentation.
# Los datos deben existir previamente 
# (se generan a partir del notebook split and augmentation)
USE_RN50 = False
DS_FLAGS = []
              # 'ref': [invertX, invertY],
              # 'rot': [rotate90, rotate180, rotate270],
              # 'crop': [crop] * CROP_TIMES,
              # 'blur': [blur],
              # 'emboss': [emboss],
              # 'randaug': [randaug],
              # 'rain': [rain],
              # 'elastic': [elastic]
                
# Las flags crop, randaug, elastic y gausblur 
# se pueden aplicar más de una vez c/u. 
# (si no están en DS_FLAGS, serán ignoradas).
CROP_TIMES = 1
RANDOM_TIMES = 1
ELASTIC_TIMES = 1
GAUSBLUR_TIMES = 1
K = 4

In [5]:
# Esta celda construye la variable data_flags, que lee DS_FLAGS de 
# la celda anterior y mapea su contenido a distintas rutas de 
# patrones, etiquetas y outputs
MAP_TIMES = {'crop': CROP_TIMES,
         'randaug': RANDOM_TIMES,
         'elastic': ELASTIC_TIMES,
         'gausblur': GAUSBLUR_TIMES
}

DS_FLAGS = sorted(DS_FLAGS)
data_flags = '_'.join(DS_FLAGS) if len(DS_FLAGS) > 0 else 'base'
MULTIPLE_TRANSF = MAP_TIMES.keys()
COPY_FLAGS = DS_FLAGS.copy()

for t in MULTIPLE_TRANSF:
    if t in DS_FLAGS:
        COPY_FLAGS.remove(t)
        COPY_FLAGS.append(t + str(MAP_TIMES[t]))
        data_flags = '_'.join(COPY_FLAGS)

# Revisión de los folds y creación de diccionario con paths
Kfolds = {}

for i in range(0, K):
    print("Fold ", i)
    patterns_dir = os.path.join(root_dir, 'patterns',  data_flags, str(i))
    labels_dir = os.path.join(root_dir, 'labels', data_flags, str(i))
    rn = 50 if USE_RN50 else 18
    models_path = os.path.join(root_dir, 'models', 'resnet', data_flags, f'resnet{rn}_K0.pth')
    features_dir = os.path.join(root_dir, 'features', 'resnet', data_flags, f'resnet{rn}_K{i}')

    if not (os.path.isdir(patterns_dir) and os.path.isdir(labels_dir)):
        print(patterns_dir)
        print(labels_dir)
        raise FileNotFoundError("""
        No existen directorios de datos para el conjunto de flags seleccionado. 
        Verifique que el dataset exista y, de lo contrario, llame a Split and Augmentation.
        """)
    if not (os.path.isfile(models_path)):
        print(models_path)
        raise FileNotFoundError(f"""
        No se encontró modelo para el conjunto de flags seleccionado. 
        Verifique que el modelo exista y, de lo contrario, llame a ResNet Retraining
        """)
                                
    Kfolds[i] = {
        'patterns_dir': patterns_dir,
        'labels_dir': labels_dir,
        'model_path': models_path,
        'features_dir': features_dir
    }
    
    print("--Pattern set encontrado en {}".format(patterns_dir))
    print("--Labels set encontrado en {}".format(labels_dir))
    print("--Modelo encontrado en {}".format(models_path))
    print("--Features a guardar en {}".format(features_dir))


Fold  0
--Pattern set encontrado en ../patterns/base/0
--Labels set encontrado en ../labels/base/0
--Modelo encontrado en ../models/resnet/base/resnet18_K0.pth
--Features a guardar en ../features/resnet/base/resnet18_K0
Fold  1
--Pattern set encontrado en ../patterns/base/1
--Labels set encontrado en ../labels/base/1
--Modelo encontrado en ../models/resnet/base/resnet18_K0.pth
--Features a guardar en ../features/resnet/base/resnet18_K1
Fold  2
--Pattern set encontrado en ../patterns/base/2
--Labels set encontrado en ../labels/base/2
--Modelo encontrado en ../models/resnet/base/resnet18_K0.pth
--Features a guardar en ../features/resnet/base/resnet18_K2
Fold  3
--Pattern set encontrado en ../patterns/base/3
--Labels set encontrado en ../labels/base/3
--Modelo encontrado en ../models/resnet/base/resnet18_K0.pth
--Features a guardar en ../features/resnet/base/resnet18_K3


## Dataset loader

In [6]:
class PatternDataset(Dataset):
    def __init__(self, root_dir, transform=None, build_classification=False, name_cla='output.cla'):
        self.root_dir = root_dir
        self.transform = transform
        self.namefiles = []
        
        self.classes = sorted(os.listdir(self.root_dir))

        for cl in self.classes:
            for pat in os.listdir(os.path.join(self.root_dir, cl)):
                self.namefiles.append((pat, cl))

        print(f'Files:{len(self.namefiles)}')
        #self.namefiles = sorted(self.namefiles, key = lambda x: x[0])
        
        if build_classification:
            dictClasses = dict()

            for cl in self.classes:
                dictClasses[cl] = []

            for index, (name, cl) in enumerate(self.namefiles):
                #print(index, name, cl)
                dictClasses[cl].append((name, index))

            with open(name_cla, 'w') as f:
                f.write('PSB 1\n')
                f.write(f'{len(self.classes)} {len(self.namefiles)}\n')
                f.write('\n')
                for cl in self.classes:
                    f.write(f'{cl} 0 {len(dictClasses[cl])}\n')
                    for item in dictClasses[cl]:
                        f.write(f'{item[1]}\n')
                    f.write('\n')

    def __len__(self):
        return len(self.namefiles)

    def __getitem__(self, index):
        if torch.is_tensor(index):
            index = index.tolist()

        img_name = os.path.join(self.root_dir, self.namefiles[index][1], self.namefiles[index][0])
        image = Image.open(img_name).convert("RGB")

        if self.transform:
            image = self.transform(image)

        return self.namefiles[index], image


## Funciones auxiliares

In [7]:
def imshow(inp, title = None):
    inp = inp.cpu().detach()
    inp = np.squeeze(inp)
    inp = inp.numpy().transpose((1, 2, 0))
    mean = np.array([0.485, 0.456, 0.406])
    std = np.array([0.229, 0.224, 0.225])
    inp = std * inp + mean
    inp = np.clip(inp, 0, 1)

    plt.imshow(inp)
    plt.show()

def get_vector(model,layer, dim_embedding, x):

  my_embedding = torch.zeros(dim_embedding)

  def copy_data(m,i,o):
    my_embedding.copy_(o.data.squeeze())

  h = layer.register_forward_hook(copy_data)
  model(x)
  h.remove()

  return my_embedding

## Extracción de features

In [8]:
import random
random.seed(30)

for i in range(0, K):
    patterns_dir = Kfolds[i]['patterns_dir']
    labels_dir = Kfolds[i]['labels_dir']
    model_path = Kfolds[i ]['model_path']
    features_dir = Kfolds[i]['features_dir']
    
    output_train = os.path.join(features_dir, "augmented_train_df.json")
    output_val = os.path.join(features_dir, "val_df.json")
    output_test = os.path.join(features_dir, "test_df.json")

    train_df = pd.read_json(os.path.join(labels_dir, "augmented_train_df.json"), orient='index')
    val_df = pd.read_json(os.path.join(labels_dir, "val_df.json"), orient='index')
    test_df = pd.read_json(os.path.join(labels_dir, "test_df.json"), orient='index')


    my_transform = transforms.Compose([ transforms.Resize(224),
                                        #transforms.CenterCrop(224),
                         transforms.ToTensor(),
                         transforms.Normalize(mean=[0.485, 0.456, 0.406],std = [0.229, 0.224, 0.225])
                        ])

    dataTrain = PatternDataset(root_dir=os.path.join(patterns_dir, 'train'), transform=my_transform)
    dataVal = PatternDataset(root_dir=os.path.join(patterns_dir, 'val'), transform=my_transform)
    dataTest = PatternDataset(root_dir=os.path.join(patterns_dir, 'test'), transform=my_transform)

    loaderTrain = DataLoader(dataTrain, shuffle="False")
    loaderVal = DataLoader(dataVal, shuffle="False")
    loaderTest = DataLoader(dataTest, shuffle="False")

    if USE_RN50:
        model = models.resnet50(pretrained = True)
    else:
        model = models.resnet18(pretrained = True)
    dim = model.fc.in_features

    output_dim = 6
    model.fc = nn.Linear(dim, output_dim)

    model = model.to(device)
    
    # Carga del modelo en models_path
    try:
        model.load_state_dict(torch.load(models_path))
    except RuntimeError as e:
        print('Ignoring "' + str(e) + '"')

    layer = model._modules.get('avgpool')

    model.eval()

    features_train = {}
    features_val = {}
    features_test = {}


    for name, img in loaderTrain:
      feat = get_vector(model, layer, dim, img.to(device))
      namefile = name[0][0]
      code, rest = namefile.split('.')
      #print(code)
      #imshow(img)
      features_train[code] = feat.numpy().tolist()
      #features.append(feat.numpy())

    for name, img in loaderVal:
      feat = get_vector(model, layer, dim, img.to(device))
      namefile = name[0][0]
      code, rest = namefile.split('.')
      #print(code)
      #imshow(img) 
      features_val[code] = feat.numpy().tolist()
      #features.append(feat.numpy())

    for name, img in loaderTest:
      feat = get_vector(model, layer, dim, img.to(device))
      namefile = name[0][0]
      code, rest = namefile.split('.')
      #print(code)
      #imshow(img)
      features_test[code] = feat.numpy().tolist()
    #features = np.vstack(features)
    #print(features.shape)

    os.makedirs(features_dir, exist_ok=True)

    features_train_df = pd.DataFrame.from_dict(features_train, orient='index')
    features_val_df = pd.DataFrame.from_dict(features_val, orient='index')
    features_test_df = pd.DataFrame.from_dict(features_test, orient='index')

    features_train_df.to_json(output_train, orient='index')
    features_val_df.to_json(output_val, orient='index')
    features_test_df.to_json(output_test, orient='index')



Files:5536
Files:78
Files:194


/home/mavergar/anaconda3/envs/Kunisch_10/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/mavergar/anaconda3/envs/Kunisch_10/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Files:5574
Files:78
Files:194


/home/mavergar/anaconda3/envs/Kunisch_10/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/mavergar/anaconda3/envs/Kunisch_10/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Files:5548
Files:78
Files:194


/home/mavergar/anaconda3/envs/Kunisch_10/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/mavergar/anaconda3/envs/Kunisch_10/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Files:5548
Files:78
Files:194


/home/mavergar/anaconda3/envs/Kunisch_10/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/mavergar/anaconda3/envs/Kunisch_10/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
